<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/447_PDO_StatsTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Test Statistical Enhancements for Proposal & Document Orchestrator

Tests statistical significance testing, confidence intervals, and trend analysis.
"""

from config import ProposalDocumentOrchestratorState, ProposalDocumentOrchestratorConfig
from agents.proposal_document_orchestrator.orchestrator import (
    create_orchestrator,
    run_orchestrator
)


def test_statistical_assessment_utilities():
    """Test statistical assessment utilities"""
    from agents.proposal_document_orchestrator.utilities.statistical_assessment import (
        assess_kpi_statistical_significance,
        assess_roi_statistical_significance,
        analyze_kpi_trends,
        calculate_statistical_assessments
    )

    # Test KPI significance (with historical data)
    historical_values = [0.85, 0.87, 0.89, 0.88, 0.90]
    current_value = 0.95

    kpi_assessment = assess_kpi_statistical_significance(
        kpi_name="document_generation_success_rate",
        current_value=current_value,
        historical_values=historical_values,
        confidence_level=0.95
    )

    assert "kpi_name" in kpi_assessment
    assert "current_value" in kpi_assessment
    assert "statistical_test" in kpi_assessment
    assert kpi_assessment["has_historical_data"] == True

    # Test KPI significance (without historical data)
    kpi_assessment_no_history = assess_kpi_statistical_significance(
        kpi_name="test_kpi",
        current_value=0.5,
        historical_values=None,
        confidence_level=0.95
    )

    assert kpi_assessment_no_history["has_historical_data"] == False
    assert "error" in kpi_assessment_no_history["statistical_test"]

    # Test ROI significance
    historical_roi = [1000, 1200, 1100, 1300, 1250]
    current_roi = 1500
    cost = 500

    roi_assessment = assess_roi_statistical_significance(
        roi_estimate=current_roi,
        cost=cost,
        historical_roi=historical_roi,
        confidence_level=0.95
    )

    assert "roi_estimate" in roi_assessment
    assert "cost" in roi_assessment
    assert "statistical_test" in roi_assessment
    assert roi_assessment["has_historical_data"] == True

    # Test trend analysis
    trend_values = [0.85, 0.87, 0.89, 0.88, 0.90, 0.92]
    trend = analyze_kpi_trends(
        kpi_name="test_trend",
        values=trend_values,
        confidence_level=0.95
    )

    assert "kpi_name" in trend
    assert "trend_analysis" in trend
    assert "trend_direction" in trend

    print("✅ test_statistical_assessment_utilities: PASSED")


def test_statistical_assessment_node():
    """Test statistical assessment node"""
    from agents.proposal_document_orchestrator.nodes import (
        goal_node,
        planning_node,
        data_loading_node,
        document_analysis_node,
        kpi_calculation_node,
        roi_calculation_node,
        statistical_assessment_node
    )
    from config import ProposalDocumentOrchestratorConfig

    config = ProposalDocumentOrchestratorConfig()

    # Build up state through previous nodes
    state: ProposalDocumentOrchestratorState = {
        "analysis_mode": "portfolio",
        "errors": []
    }

    state = {**state, **goal_node(state)}
    state = {**state, **planning_node(state)}
    state = {**state, **data_loading_node(state, config)}
    state = {**state, **document_analysis_node(state)}
    state = {**state, **kpi_calculation_node(state, config)}
    state = {**state, **roi_calculation_node(state, config)}

    # Test statistical assessment node
    stats_result = statistical_assessment_node(state, config)
    state = {**state, **stats_result}

    assert "errors" in state
    assert len(state["errors"]) == 0, f"Errors: {state['errors']}"
    assert "statistical_assessments" in state
    assert "confidence_intervals" in state
    assert "trends" in state

    # Verify statistical assessments structure
    stats = state["statistical_assessments"]
    assert "kpi_assessments" in stats
    assert "roi_assessment" in stats

    print("✅ test_statistical_assessment_node: PASSED")


def test_statistical_section_in_report():
    """Test that report includes statistical assessment section"""
    from agents.proposal_document_orchestrator.utilities.report_generation import (
        generate_executive_report,
        generate_statistical_assessment_section
    )
    from agents.proposal_document_orchestrator.nodes import (
        goal_node,
        planning_node,
        data_loading_node,
        document_analysis_node,
        kpi_calculation_node,
        roi_calculation_node,
        workflow_analysis_node,
        portfolio_summary_node,
        statistical_assessment_node
    )
    from config import ProposalDocumentOrchestratorConfig

    config = ProposalDocumentOrchestratorConfig()

    # Build up state through all nodes
    state: ProposalDocumentOrchestratorState = {
        "analysis_mode": "portfolio",
        "errors": []
    }

    state = {**state, **goal_node(state)}
    state = {**state, **planning_node(state)}
    state = {**state, **data_loading_node(state, config)}
    state = {**state, **document_analysis_node(state)}
    state = {**state, **kpi_calculation_node(state, config)}
    state = {**state, **roi_calculation_node(state, config)}
    state = {**state, **workflow_analysis_node(state, config)}
    state = {**state, **portfolio_summary_node(state)}
    state = {**state, **statistical_assessment_node(state, config)}

    # Generate report
    report = generate_executive_report(state, config)

    # Check for statistical section
    assert "Statistical Assessment" in report

    # Test statistical section generation directly
    stats_section = generate_statistical_assessment_section(
        state["statistical_assessments"],
        state["confidence_intervals"]
    )

    assert len(stats_section) > 0
    assert "Statistical Assessment" in stats_section

    print("✅ test_statistical_section_in_report: PASSED")


def test_end_to_end_with_statistics():
    """Test complete end-to-end workflow with statistical assessment"""
    from config import ProposalDocumentOrchestratorConfig

    config = ProposalDocumentOrchestratorConfig()

    # Ensure statistical testing is enabled
    config.enable_statistical_testing = True

    # Run orchestrator
    final_state = run_orchestrator(
        analysis_mode="portfolio",
        config=config
    )

    # Verify no errors
    errors = final_state.get("errors", [])
    assert len(errors) == 0, f"Errors in end-to-end workflow: {errors}"

    # Verify statistical assessments are present
    assert "statistical_assessments" in final_state

    # confidence_intervals can be at top level OR inside statistical_assessments
    # (LangGraph state merging may place it in either location)
    stats = final_state.get("statistical_assessments", {})
    has_ci_top = "confidence_intervals" in final_state
    has_ci_inside = "confidence_intervals" in stats

    assert has_ci_top or has_ci_inside, (
        f"confidence_intervals not found. Top level: {has_ci_top}, "
        f"Inside stats: {has_ci_inside}. Stats keys: {list(stats.keys())}"
    )

    # Store reference for later use
    confidence_intervals = final_state.get("confidence_intervals") or stats.get("confidence_intervals", {})
    assert isinstance(confidence_intervals, dict), "confidence_intervals should be a dict"

    assert "trends" in final_state

    # Verify report includes statistical section
    report = final_state.get("executive_report", "")
    assert "Statistical Assessment" in report

    # Verify statistical assessments structure
    stats = final_state["statistical_assessments"]
    assert "kpi_assessments" in stats
    assert "roi_assessment" in stats

    print("✅ test_end_to_end_with_statistics: PASSED")


def test_statistical_assessment_without_historical_data():
    """Test that statistical assessment works gracefully without historical data"""
    from agents.proposal_document_orchestrator.nodes import (
        goal_node,
        planning_node,
        data_loading_node,
        document_analysis_node,
        kpi_calculation_node,
        roi_calculation_node,
        statistical_assessment_node
    )
    from config import ProposalDocumentOrchestratorConfig

    config = ProposalDocumentOrchestratorConfig()
    config.enable_statistical_testing = True

    # Build up state (no historical data)
    state: ProposalDocumentOrchestratorState = {
        "analysis_mode": "portfolio",
        "errors": []
    }

    state = {**state, **goal_node(state)}
    state = {**state, **planning_node(state)}
    state = {**state, **data_loading_node(state, config)}
    state = {**state, **document_analysis_node(state)}
    state = {**state, **kpi_calculation_node(state, config)}
    state = {**state, **roi_calculation_node(state, config)}

    # Test statistical assessment node (should work without historical data)
    stats_result = statistical_assessment_node(state, config)
    state = {**state, **stats_result}

    # Should not have errors (graceful handling of no historical data)
    assert "errors" in state
    # May have warnings but should not fail

    # Should still have statistical assessments structure
    assert "statistical_assessments" in state

    print("✅ test_statistical_assessment_without_historical_data: PASSED")


if __name__ == "__main__":
    print("Testing Statistical Enhancements for Proposal & Document Orchestrator\n")

    try:
        test_statistical_assessment_utilities()
        test_statistical_assessment_node()
        test_statistical_section_in_report()
        test_end_to_end_with_statistics()
        test_statistical_assessment_without_historical_data()

        print("\n✅ All statistical enhancement tests passed!")
    except Exception as e:
        print(f"\n❌ Test failed: {e}")
        import traceback
        traceback.print_exc()
        exit(1)


In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_013_Proposal&Document_Orchestrator % python3 test_proposal_document_orchestrator_statistical.py
Testing Statistical Enhancements for Proposal & Document Orchestrator

✅ test_statistical_assessment_utilities: PASSED
✅ test_statistical_assessment_node: PASSED
✅ test_statistical_section_in_report: PASSED
✅ test_end_to_end_with_statistics: PASSED
✅ test_statistical_assessment_without_historical_data: PASSED

✅ All statistical enhancement tests passed!
